# INE: Población por sexo, municipio, nacionalidad y edad

In [2]:
import requests
import pandas as pd

In [3]:
# código de provincias  01 ... 51
codigo_provincias = [format(n, '02d') for n in range(1, 52)]

url_base = 'http://www.ine.es/jaxi/files/_px/es/xls/t20/e245/p05/a2016/l0/000{}002.px?'
url_provincias = [url_base.format(c) for c in codigo_provincias]

### Ejemplo con Burgos

In [4]:
# Provincia de burgos
codigo_provincia = codigo_provincias[8]
url_provincia = url_provincias[8]

# Carga el excel sin las primeras líneas ni las últimas
#df = pd.read_excel(url_provincia, skiprows=6, skip_footer=6, header=[0, 1], squeeze=False)
df = pd.read_excel(url_provincia, skip_footer=6)

# Guarda la cabecera para cambiar el nombre de las columnas
cabecera = df.iloc[5]
provincia = df.iloc[0, 0]
df.columns = df.iloc[6]
# Elimina la cabecera
df = df[7:]

# Da un nombre a la primera columna
df.columns.values[0] = 'Municipio'

# Elimina los espacios antes del municipio
df['Municipio'] = df['Municipio'].str.strip()


df.head()

6,Municipio,Total,Menores de 16 años,De 16 a 64 años,De 65 y más años,Total,Menores de 16 años,De 16 a 64 años,De 65 y más años,Total,Menores de 16 años,De 16 a 64 años,De 65 y más años
7,Ambos sexos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Total,360995,50324,228655,82016,335320,45941,207877,81502,25675,4383,20778,514
9,09001-Abajas,29,0,15,14,28,0,14,14,1,0,1,0
10,09003-Adrada de Haza,222,19,132,71,211,15,125,71,11,4,7,0
11,09006-Aguas Cándidas,58,2,31,25,58,2,31,25,0,0,0,0


In [5]:
# Filas que empiezan por el código de provincia
subtotales = df.loc[~df['Municipio'].str.startswith(codigo_provincia)]
# Filas que además no contienen 'Total'
subtotales = subtotales.loc[~subtotales['Municipio'].str.contains('Total')]

sub_nombre = subtotales['Municipio'].values
sub_index = subtotales['Municipio'].index

sub_nombre, sub_index

(array(['Ambos sexos', 'Hombres', 'Mujeres'], dtype=object),
 Int64Index([7, 380, 753], dtype='int64'))

In [6]:
# Subtotales
# 0: ambos sexos, 1: hombres, 2: mujeres
sub_data = []

for i in range(len(sub_nombre)):
    ini = sub_index[i] + 1
    fin = ini + (sub_index[1] - sub_index[0])
    sub_data.append(df[(df.index > ini) & (df.index < fin - 1)])

sub_data[0].head()

6,Municipio,Total,Menores de 16 años,De 16 a 64 años,De 65 y más años,Total,Menores de 16 años,De 16 a 64 años,De 65 y más años,Total,Menores de 16 años,De 16 a 64 años,De 65 y más años
9,09001-Abajas,29,0,15,14,28,0,14,14,1,0,1,0
10,09003-Adrada de Haza,222,19,132,71,211,15,125,71,11,4,7,0
11,09006-Aguas Cándidas,58,2,31,25,58,2,31,25,0,0,0,0
12,09007-Aguilar de Bureba,64,1,37,26,59,0,33,26,5,1,4,0
13,09009-Albillos,213,45,141,27,212,45,140,27,1,0,1,0


In [7]:
# Mezcla las dos cabeceras en una sola

def new_column_names(header, nombre_subtotal, cabecera):
    # La primera columna (municipio) solo aparece una vez sin modicarse
    columnas = [header[0]]
    anterior = None
    
    for nombre, tipo in zip(header[1:], cabecera[1:]):
        if type(tipo) is str:
            anterior = tipo
        else:
            tipo = anterior
        nuevo_nombre = '-'.join([nombre_subtotal, tipo, nombre])
        columnas.append(nuevo_nombre)
    
    return columnas

In [8]:
# Cambia las columnas de cada subdata

for data, nombre, index in zip(sub_data, sub_nombre, sub_index):
    new_columns = new_column_names(data.columns, nombre, cabecera)
    data.columns = new_columns

In [8]:
sub_data[0].head()

,Municipio,Ambos sexos-Total-Total,Ambos sexos-Total-Menores de 16 años,Ambos sexos-Total-De 16 a 64 años,Ambos sexos-Total-De 65 y más años,Ambos sexos-españoles-Total,Ambos sexos-españoles-Menores de 16 años,Ambos sexos-españoles-De 16 a 64 años,Ambos sexos-españoles-De 65 y más años,Ambos sexos-extranjeros-Total,Ambos sexos-extranjeros-Menores de 16 años,Ambos sexos-extranjeros-De 16 a 64 años,Ambos sexos-extranjeros-De 65 y más años
9,09001-Abajas,29,0,15,14,28,0,14,14,1,0,1,0
10,09003-Adrada de Haza,222,19,132,71,211,15,125,71,11,4,7,0
11,09006-Aguas Cándidas,58,2,31,25,58,2,31,25,0,0,0,0
12,09007-Aguilar de Bureba,64,1,37,26,59,0,33,26,5,1,4,0
13,09009-Albillos,213,45,141,27,212,45,140,27,1,0,1,0


In [9]:
# Une todos los sub_data
data = sub_data[0]

for s_data in sub_data[1:]:
    # Une los sub_data
    data = data.merge(s_data, on=['Municipio'])

# Añade la columna Provincia al principio del dataframe final
data.insert(0, 'Provincia', provincia)

# Elimina el código de provincia dejando sólo el nombre
data['Provincia'] = data['Provincia'].str.split('.-').str[-1].str.strip()
# Elimina el código del municipio
data['Municipio'] = data['Municipio'].str.split('-').str[-1]


data.head()

,Provincia,Municipio,Ambos sexos-Total-Total,Ambos sexos-Total-Menores de 16 años,Ambos sexos-Total-De 16 a 64 años,Ambos sexos-Total-De 65 y más años,Ambos sexos-españoles-Total,Ambos sexos-españoles-Menores de 16 años,Ambos sexos-españoles-De 16 a 64 años,Ambos sexos-españoles-De 65 y más años,...,Mujeres-Total-De 16 a 64 años,Mujeres-Total-De 65 y más años,Mujeres-españoles-Total,Mujeres-españoles-Menores de 16 años,Mujeres-españoles-De 16 a 64 años,Mujeres-españoles-De 65 y más años,Mujeres-extranjeros-Total,Mujeres-extranjeros-Menores de 16 años,Mujeres-extranjeros-De 16 a 64 años,Mujeres-extranjeros-De 65 y más años
0,Burgos,Abajas,29,0,15,14,28,0,14,14,...,4,6,9,0,3,6,1,0,1,0
1,Burgos,Adrada de Haza,222,19,132,71,211,15,125,71,...,60,39,103,8,56,39,5,1,4,0
2,Burgos,Aguas Cándidas,58,2,31,25,58,2,31,25,...,7,16,23,0,7,16,0,0,0,0
3,Burgos,Aguilar de Bureba,64,1,37,26,59,0,33,26,...,14,14,26,0,12,14,3,1,2,0
4,Burgos,Albillos,213,45,141,27,212,45,140,27,...,60,13,97,25,59,13,1,0,1,0


# Completo

In [11]:
# Mezcla las dos cabeceras en una sola
def new_column_names(header, nombre_subtotal, cabecera):
    # La primera columna (municipio) solo aparece una vez sin modicarse
    columnas = [header[0]]
    anterior = None

    for nombre, tipo in zip(header[1:], cabecera[1:]):
        if type(tipo) is str:
            anterior = tipo
        else:
            tipo = anterior
        nuevo_nombre = '-'.join([nombre_subtotal, tipo, nombre])
        columnas.append(nuevo_nombre)

    return columnas

In [3]:
def procesa_provincia(codigo_provincia, url_provincia):
    # Carga el excel sin las primeras líneas ni las últimas
    df = pd.read_excel(url_provincia, skip_footer=6)

    # Guarda la cabecera para cambiar el nombre de las columnas
    cabecera = df.iloc[5]
    provincia = df.iloc[0, 0]
    df.columns = df.iloc[6]
    # Elimina la cabecera
    df = df[7:]

    # Da un nombre a la primera columna
    df.columns.values[0] = 'Municipio'

    # Filas que empiezan por el código de provincia
    subtotales = df.loc[~df['Municipio'].str.startswith('    ' + codigo_provincia)]
    # Filas que además no contienen 'Total'
    subtotales = subtotales.loc[~subtotales['Municipio'].str.contains('Total')]

    sub_nombre = subtotales['Municipio'].values
    sub_index = subtotales['Municipio'].index

    # Subtotales
    # 0: ambos sexos, 1: hombres, 2: mujeres
    sub_data = []

    
    for i in range(len(sub_nombre)):
        ini = sub_index[i] + 1
        fin = ini + (sub_index[1] - sub_index[0])
        sub_data.append(df[(df.index > ini) & (df.index < fin - 1)])

        
    # Cambia las columnas de cada subdata
    for data, nombre, index in zip(sub_data, sub_nombre, sub_index):
        new_columns = new_column_names(data.columns, nombre, cabecera)
        data.columns = new_columns
        
    # Une todos los sub_data
    data = sub_data[0]

    for s_data in sub_data[1:]:
        # Une los sub_data
        data = data.merge(s_data, on=['Municipio'])

    # Añade la columna Provincia al principio del dataframe final
    data.insert(0, 'Provincia', provincia)

    return data

In [13]:
# código de provincias  01 ... 51
codigo_provincias = [format(n, '02d') for n in range(1, 52)]

url_base = 'http://www.ine.es/jaxi/files/_px/es/xls/t20/e245/p05/a2016/l0/000{}002.px?'
url_provincias = [url_base.format(c) for c in codigo_provincias]

datos_provincias = []

for provincia in zip(codigo_provincias, url_provincias):
    datos_provincia = procesa_provincia(*provincia)
    datos_provincias.append(datos_provincia)

# Dataframe con todas las provincias
datos = pd.concat(datos_provincias)

# Elimina el código de provincia dejando sólo el nombre
datos['Provincia'] = datos['Provincia'].str.split('.-').str[-1].str.strip()
# Elimina el código del municipio
datos['Municipio'] = datos['Municipio'].str.split('-').str[-1]

# Elimina el índice que se ha generado al juntar los dataframes
datos = datos.reset_index(drop=True)

#datos.to_excel("Todas_Provincias.xls")

,Provincia,Municipio,Ambos sexos-Total-Total,Ambos sexos-Total-Menores de 16 años,Ambos sexos-Total-De 16 a 64 años,Ambos sexos-Total-De 65 y más años,Ambos sexos-españoles-Total,Ambos sexos-españoles-Menores de 16 años,Ambos sexos-españoles-De 16 a 64 años,Ambos sexos-españoles-De 65 y más años,...,Mujeres-Total-De 16 a 64 años,Mujeres-Total-De 65 y más años,Mujeres-españoles-Total,Mujeres-españoles-Menores de 16 años,Mujeres-españoles-De 16 a 64 años,Mujeres-españoles-De 65 y más años,Mujeres-extranjeros-Total,Mujeres-extranjeros-Menores de 16 años,Mujeres-extranjeros-De 16 a 64 años,Mujeres-extranjeros-De 65 y más años
0,Araba/Álava,Dulantzi,2856,694,1875,287,2632,625,1721,286,...,917,148,1281,292,842,147,108,32,75,1
1,Araba/Álava,Amurrio,10260,1530,6658,2072,9655,1399,6198,2058,...,3293,1137,4866,665,3073,1128,299,70,220,9
2,Araba/Álava,Añana,159,9,100,50,151,7,94,50,...,32,34,67,4,29,34,4,1,3,0
3,Araba/Álava,Aramaio,1502,265,935,302,1476,263,911,302,...,433,146,699,130,423,146,11,1,10,0
4,Araba/Álava,Armiñón,234,46,157,31,223,45,147,31,...,76,17,108,20,71,17,5,0,5,0


In [1]:
datos.head()

NameError: name 'datos' is not defined